### 2.3 The gears of neural networks: tensor operations

Much as any computer program can be ultimately reduced to a small set of binary operations on binary inputs (AND, OR, NOR, and so on), all transformations learned by deep neural networks can be reduced to a handful of *tensor operations* applied to tensors of numeric data. For instance, it's possible to add tensors, multiply tensors, and so on.
In our initial example, we were building our network by stacking `Dense` layers on top of each other. A keras layer instance looks like this:

`tf.keras.layers.Dense(512, activation='relu')`

This layer can be interpreted as a function, which takes as input a 2D tensor and returns another 2D tensor, a new representation for the input tensor. Specifically, the function is as follows (where `W` is a 2D tensor and `b` is a vector, both attributes of the layer):

`output = relu(dot(W, input) + b)`

Let's unpack this. We have three tensor operations here: a dot product (`dot`) between the input tensor and a tensor named `W`; an addition (`+`) between the resulting 2D tensor and a vector `b`; and, finally, a `relu` operation. `relu(x)` is `max(x, 0)`.
    
**NOTE** Although this section deals entirely with linear algebra expression, you won't find any mathematical notation here. I've found that mathematical concepts can be more readily mastered by programmers with no mathematical background if they're expressed as short Python snippets instead of mathematical equations. So  we'll use Numpy code throughout.

#### 2.3.1 Element-wise operations

The `relu` operation and addition are *element-wise* operations: operations that are applied independently to each entry in the tensors being considered. This means these operations are highly amenable to massively parallel implementations. If you want to write a naive Python implementation of an element-wise operation, you use a `for` loop, as in this naive implementation of an element-wise `relu` operation:

In [1]:
def naive_relu(x):
    assert len(x.shape) == 2
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

tensor = np.random.random((1, 1))
print(f"Naive Relu of value {tensor}: {naive_relu(tensor)}")

NameError: name 'np' is not defined

You do the same for addition:

In [ ]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

On the same principle, you can do element-wise multiplication, subtraction, and so on. 
In practice, when dealing with Numpy arrays, these operations are available as well-optimized built-in Numpy functions, which themselves delegate the heavy lifting to a Basic Linear Algebra Subprograms (BLAS) implementation if you have one installed. BLAS are low-level, highly parallel, efficient tensor-manipulation routines that are typically implemented in Fortran or C.

So, in Numpy, you can do the following element-wise operation, and it will be blazing fast:

```
import numpy as np
z = x + y
z = np.maximum(z, 0.)
```

#### 2.3.2 Broadcasting

Our earlier naive implementation of *naive_add* only supports the addition of 2D tensors with identical shapes. But in the `Dense` layer introduced earlier, we added a 2D tensor with a vector. What happens with addition when the shapes of the two tensors being added differ?
When possible, and if there's no ambiguity, the smaller tensor will be *broadcasted* to match the shape of the larger tensor. Broadcasting consists of two steps:
1. Axes (called *broadcast axes*) are added to the smaller tensor to match the `ndim` of the larger tensor.
2. The smaller tensor is repreated alongside these new axes to match the full shape of the larger tensor.

Let's look at a concrete example. Consider `X` with shape (32, 10) and `y` with shape (10,). First, we add an empty first axis to `y`, whose shape becomes (1, 10). Then, we repeat `y` 32 times alongside this new axis, so that we end up with a tenor `Y` with shape (32, 10), where Y[i, :] == y for i in range (0, 32). At this point, we can proceed to add X and Y, because they have the same shape.
In terms of implementation, no new 2D tensor is created, because that would be terribly inefficient. The repetition operation is entirely virtual: it happens at the algorithmic level rather than at the memory level. But thinking of the vector being repeated 10 times alongside a new axis is a helpful mental model. Here's what a naive implementation would look like: 

In [ ]:
# Self added Example

# Import Numpy
import numpy as np

# Get a rank 2 tensor of shape (32, 10)
X = np.random.random((32, 10))
print(f"Shape of X: {X.shape}")

# Get a rank 1 tensor of shape (10,)
y = np.random.random((10,))
print(f"Shape of y: {y.shape}")

# Add a new axis to smaller tensor
y = np.expand_dims(y, axis=0)
print(f"Added a new axis to y: {y.shape}")

# Repeat 'y' along the new axis X.shape[0] (32) times
Y = np.repeat(y, X.shape[0], axis=0)
print(f"Shape of Y: {Y.shape}")


In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

With broadcasting you can generally apply two-tensor element-wise operations if one tensor has shape `(a, b, ... n, n+1, ... m)` and the other has shape `(n, n + 1, ... m)`. The broadcasting will then automatically happen for axes `a` through `n - 1`.
The following example applies the element-wise `maximum` operation to two tensors of different shapes via broadcasting:

In [ ]:
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))
z = np.maximum(x, y)
print(f"Maximum tensor: {z.shape}")

#### 2.3.3 Tensor dot

The dot operation, also called a *tensor product* (not to be confused with an element-wise product) is the most common, most useful tensor operation. Contrary to element-wise operations, it combines entries in the input tensors.
An element-wise product is done with the * operator in Numpy, Keras, Theano and Tensorflow. `dot` uses a different syntax in Tensorflow, but in both Numpy and Keras it's done using the standard `dot` operator:

```
z = np.dot(x, y)
```

In mathematical notation, you'd note the operation with a dot(.):

```
z = x . y
```

Mathematically, what does the dot operation do? Let's start with the dot product of two vectors x and y. It's computed as follows:

In [ ]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return 0

You'll have noticed that the dot product between two vectors is a scalar and that only vectors with the same number of elements are compatible for a dot product.
You can also take the dot product between a matrix `x` and a vector `y`, which returns a vector where the coefficients are the dot products between `y` and the rows of x. You implement it as follows.

In [ ]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

You could also reuse the code we wrote previously, which highlights the relationship between a matrix-vector product and a vector product:

In [ ]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

Note that as soon as one of the two tensors has an `ndim` greater than 1, `dot` is no longer symmetric, which is to say that `dot(x, y)` isn't the same as `dot(y, x)`.
Of course, a dot product generalizes to tensors with an arbitrary number of axes. The most common applications may be the dot product between two matrices. You can take the dot product of two matrices `x` and `y` (`dot(x, y)`) if and only if `x.shape[0] == y.shape[0]`. The result is a matrix with shape (`x.shape[0]`, `y.shape[1]`), where the coefficients are the vector products between the rows of `x` and the columns of `y`. Here's the naive implementation: